In [4]:
import re
from seam import nino34, utils
from concurrent.futures import ProcessPoolExecutor

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# List of CESM2 Ensemble Members
members = ['LE2-1001.001', 'LE2-1011.001', 'LE2-1021.002', 'LE2-1031.002', 'LE2-1041.003', 'LE2-1051.003',
            'LE2-1061.004', 'LE2-1071.004', 'LE2-1081.005', 'LE2-1091.005', 'LE2-1101.006', 'LE2-1111.006',
            'LE2-1121.007', 'LE2-1131.007', 'LE2-1141.008', 'LE2-1151.008', 'LE2-1161.009', 'LE2-1171.009',
            'LE2-1181.010', 'LE2-1191.010', 
            'LE2-1231.001', 'LE2-1231.002', 'LE2-1231.003', 'LE2-1231.004',
            'LE2-1231.005', 'LE2-1231.006', 'LE2-1231.007', 'LE2-1231.008', 'LE2-1231.009', 'LE2-1231.010',
            'LE2-1231.011', 'LE2-1231.012', 'LE2-1231.013', 'LE2-1231.014', 'LE2-1231.015', 'LE2-1231.016',
            'LE2-1231.017', 'LE2-1231.018', 'LE2-1231.019', 'LE2-1231.020',
            'LE2-1251.001', 'LE2-1251.002', 'LE2-1251.003', 'LE2-1251.004',
            'LE2-1251.005', 'LE2-1251.006', 'LE2-1251.007', 'LE2-1251.008', 'LE2-1251.009', 'LE2-1251.010',
            'LE2-1251.011', 'LE2-1251.012', 'LE2-1251.013', 'LE2-1251.014', 'LE2-1251.015', 'LE2-1251.016',
            'LE2-1251.017', 'LE2-1251.018', 'LE2-1251.019', 'LE2-1251.020',
            'LE2-1301.001', 'LE2-1301.002', 'LE2-1301.003', 'LE2-1301.004', 'LE2-1301.005', 'LE2-1301.006',
            'LE2-1301.007', 'LE2-1301.008', 'LE2-1301.009', 'LE2-1301.010', 'LE2-1301.011', 'LE2-1301.012', 
            'LE2-1301.013', 'LE2-1301.014', 'LE2-1301.015', 'LE2-1301.016', 'LE2-1301.017', 'LE2-1301.018',
            'LE2-1301.019', 'LE2-1301.020',
            'LE2-1281.001', 'LE2-1281.002', 'LE2-1281.003', 'LE2-1281.004', 'LE2-1281.005', 'LE2-1281.006',
            'LE2-1281.007', 'LE2-1281.008', 'LE2-1281.009', 'LE2-1281.010', 'LE2-1281.011', 'LE2-1281.012',
            'LE2-1281.013', 'LE2-1281.014', 'LE2-1281.015', 'LE2-1281.016', 'LE2-1281.017', 'LE2-1281.018',
            'LE2-1281.019', 'LE2-1281.020']

In [2]:

# Define directories and filename patterns
SST_dir = "/net/fs01/data/CESM2_projects/CESM2-LE/ocn/monthly_ave/SST/"
PRECT_dir = "/net/fs01/data/CESM2_projects/CESM2-LE/atm/monthly_ave/PRECT/"
SST_pattern = "b.e21.*.f09_g17.{}.pop.h.SST.*.nc"
PRECT_pattern = "b.e21.*.f09_g17.{}.cam.h0.PRECT.*.nc"

# Code to pre-process CESM2-LE data. 
# In Figure 1, we use timeseries of DJF mean global SSTs from 1900-2100 for each ensemble member ("{ens}.SST.global.1900_2100.nc")
# In Figure 1, we also use timeseries of monthly precip. in MSEA from 1900-2100 for each ensemble member ("{ens}.PRECT.MSEA.nc")
# In Figure 2,we use timeseries of monthly Niño3.4 SSTs from 1900-2100 for each ensemble members ("{ens}.SST.Nino34.nc")")

def process_member(ens):
    print(f"Processing {ens}")
    SST_file = re.compile(SST_pattern.format(ens))
    PRECT_file = re.compile(PRECT_pattern.format(ens))

    SST_file_list = utils.get_file_list(SST_dir, SST_file)
    ds0 = utils.get_multifile_ds_cesm(SST_file_list, variable='SST')
    sst_ds = ds0.sel(time=slice("1900-01", "2100-12"))
    sst_da = sst_ds["SST"]
    sst_slice = nino34.get_nino34_area(sst_da)

    #sst_slice.to_netcdf(f"/home/eleroy/proj-dirs/SEAM/data/analysis_data/CESM-LE2_cropped_data/{ens}.SST.Nino34.nc")


    PRECT_file_list = utils.get_file_list(PRECT_dir, PRECT_file)
    ds0 = utils.get_multifile_ds_cesm(PRECT_file_list, variable='PRECT')
    precip_ds = ds0.sel(time=slice("1900-01", "2100-12"))
    precip_da = precip_ds["PRECT"]
    
    utils.check_data_conventions(precip_da)
    
    # Select MSEA region (90, 110, 10, 25)
    maxlat = 25
    minlat = 10
    minlon = 90
    maxlon = 110

    prec_da = precip_da.sel(
        lat=slice(minlat, maxlat), lon=slice(minlon, maxlon)
    )

    #prec_da.to_netcdf(f"/home/eleroy/proj-dirs/SEAM/data/analysis_data/CESM-LE2_cropped_data/{ens}.PRECT.MSEA.nc")

def main(members):
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(process_member, members))

    
#main(members) #uncomment to run code

In [ ]:
def process_member(ens):
    print(f"Processing {ens}")
    SST_file = re.compile(SST_pattern.format(ens))
    SST_file_list = utils.get_file_list(SST_dir, SST_file)
    ds0 = utils.get_multifile_ds_cesm(SST_file_list, variable='SST')
    sst_ds = ds0 #.sel(time=slice("1951-01", "2015-12"))
    sst_da = sst_ds["SST"]
    
    sst_anm0 = sst_da.resample(time='QS-DEC', label='left').mean()
    sst_anm = sst_anm0.sel(time=sst_anm0.time.dt.month.isin([12])).resample(time='1Y').mean()

    #sst_anm.to_netcdf(f"/home/eleroy/proj-dirs/SEAM/data/analysis_data/CESM-LE2_cropped_data/{ens}.SST.global.1900_2100.nc")


def main(members):

    with ProcessPoolExecutor() as executor:
        results = list(executor.map(process_member, members))


#ssts = main(members)

In [ ]:
# Figure 4 pre-processing 

# Commented out code to save precip data separately (labeled "MSEA" but these are global)
# Define directories and filename patterns
PRECT_dir = "/net/fs01/data/CESM2_projects/CESM2-LE/atm/monthly_ave/PRECT/"
PRECT_pattern = "b.e21.*.f09_g17.{}.cam.h0.PRECT.*.nc"

def process_member(ens):
    print(f"Processing {ens}")
    PRE_file = re.compile(PRECT_pattern.format(ens))
    PRE_file_list = utils.get_file_list(PRECT_dir, PRE_file)
    ds0 = utils.get_multifile_ds_cesm(PRE_file_list, variable='PRECT')
    pre_ds = ds0 #.sel(time=slice("1951-01", "2015-12"))
    pre_da = pre_ds["PRECT"]
    
    pre_anm0 = pre_da.resample(time='QS-DEC', label='left').mean()
    pre_anm = pre_anm0.sel(time=pre_anm0.time.dt.month.isin([3])).resample(time='1Y').mean()

    #pre_anm.to_netcdf(f"/home/eleroy/proj-dirs/SEAM/data/analysis_data/CESM-LE2_cropped_data/{ens}.PRECT.msea.MAM.1900_2100.nc")


#def main(members):

#    with ProcessPoolExecutor() as executor:
#        results = list(executor.map(process_member, members))


#pres = main(members)

# Define directories and filename patterns
PSL_dir = "/net/fs01/data/CESM2_projects/CESM2-LE/atm/monthly_ave/PSL/"
PSL_pattern = "b.e21.*.f09_g17.{}.cam.h0.PSL.*.nc"

def process_member(ens):
    print(f"Processing {ens}")
    PSL_file = re.compile(PSL_pattern.format(ens))
    PSL_file_list = utils.get_file_list(PSL_dir, PSL_file)
    ds0 = utils.get_multifile_ds_cesm(PSL_file_list, variable='PSL')
    psl_ds = ds0 #.sel(time=slice("1951-01", "2015-12"))
    psl_da = psl_ds["PSL"]
    
    psl_anm = psl_da.resample(time='1Y').mean()

    psl_anm0 = psl_da.resample(time='QS-DEC', label='left').mean()
    psl_anm = psl_anm0.sel(time=psl_anm0.time.dt.month.isin([12])).resample(time='1Y').mean()

    #psl_anm.to_netcdf(f"/home/eleroy/proj-dirs/SEAM/data/analysis_data/CESM-LE2_cropped_data/{ens}.PSL.global.DJF.1900_2100.nc")


    #psl_anm.to_netcdf(f"/home/eleroy/proj-dirs/SEAM/data/analysis_data/CESM-LE2_cropped_data/{ens}.PSL.global.annual.1900_2100.nc")


def main(members):

    with ProcessPoolExecutor() as executor:
        results = list(executor.map(process_member, members))


psl = main(members)